In [1]:
import tensorflow as tf
import pickle
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
final_best_model = load_model('best_model.keras')
tokenizer_object = 'tokenizer.pkl'
with open(tokenizer_object, 'rb') as file:
    tokenizer = pickle.load(file)

maxlength_object = 'max_length.pkl'
with open(maxlength_object, 'rb') as file:
    maxlength_object = pickle.load(file)

threshold_object = 'best_threshold.pkl'
with open(threshold_object, 'rb') as file:
    threshold_object = pickle.load(file)

In [3]:
# Long complex movie reviews that delve into nuanced opinions and perspectives
movie_reviews = [
    "The film presents a deeply layered narrative that intertwines themes of loss, redemption, and human connection. While the pacing was deliberate, it allowed the emotional weight of each scene to resonate with the audience. The cinematography was breathtaking, capturing the stark beauty of the landscapes and the raw intensity of the characters' emotions. However, the overly ambiguous ending left some viewers dissatisfied, as it raised more questions than it answered.",
    "A bold attempt at exploring societal issues through a dystopian lens, the movie falls short in execution. The script tries to juggle too many subplots, leaving several unresolved by the end. While the lead actor delivers a powerful performance, the supporting cast felt underutilized, and the soundtrack, though haunting, sometimes overshadowed critical dialogue. Nonetheless, the film's ambition deserves commendation, even if the result was uneven.",
    "This is not merely a story; it's an experience. The director meticulously crafts every frame to evoke a sense of nostalgia and longing. The subtle interplay between light and shadow adds depth to the narrative, while the characters' imperfections make them utterly relatable. It might not appeal to those seeking high-octane action, but for viewers craving meaningful storytelling, it's a treasure trove.",
    "An extraordinary blend of horror and psychological drama, the film toys with the audience's perception of reality. Its slow-burning tension builds to a crescendo in the final act, leaving viewers stunned and introspective. The symbolism woven into the narrative invites multiple interpretations, turning the movie into a puzzle that lingers in your mind long after the credits roll. However, its complexity might alienate casual moviegoers.",
    "The film boasts a star-studded cast and lush production design, but unfortunately, it leans too heavily on spectacle over substance. While visually stunning, the narrative lacked emotional resonance, and the dialogues often felt contrived. The pacing was inconsistent, and the plot twists, though unexpected, failed to leave a lasting impact. It's an entertaining watch but doesn't rise above mediocrity.",
    "A heartfelt exploration of family dynamics and intergenerational conflict, the movie succeeds in portraying the complexity of human relationships. The screenplay was deeply moving, supported by stellar performances from the cast. The musical score complemented the emotional beats perfectly, enhancing the overall experience. Despite its brilliance, some might find the narrative overly sentimental at times.",
    "Brilliantly executed with masterful precision, the film pushes the boundaries of storytelling. Every element—be it the plot, characters, or cinematography—works cohesively to create a deeply immersive experience. The layers of meaning embedded in the script challenge the audience to think critically, making this film a rare gem in modern cinema. However, its unorthodox approach may polarize viewers.",
    "Although the premise holds promise, the execution is far from perfect. The pacing is erratic, and the character arcs feel rushed, leaving little room for emotional engagement. The visuals are undoubtedly captivating, and the soundtrack elevates certain scenes, but the lack of coherence in the storytelling undermines the overall impact. It’s a film that has potential but fails to capitalize on it.",
    "A cinematic marvel that breathes life into a well-worn genre. The dialogues are sharp and witty, and the performances from the ensemble cast are nothing short of spectacular. The director's nuanced handling of sensitive themes deserves praise, as does the seamless integration of the musical score into the narrative. While nearly flawless, the third act drags a bit, leaving a slight blemish on an otherwise stellar film.",
    "The movie dives headfirst into exploring the human psyche, unraveling profound existential questions. Its daring approach to storytelling is matched by its striking visuals and brilliant performances. The nonlinear narrative may confuse some viewers, but those willing to engage with its complexity will find it deeply rewarding. It's a bold cinematic achievement that isn't afraid to challenge conventions."
]

In [4]:
new_reviews_sequences = tokenizer.texts_to_sequences(movie_reviews)
padded_new_reviews = pad_sequences(new_reviews_sequences, padding='post', truncating='post', maxlen=maxlength_object)

In [5]:
predictions = final_best_model.predict(padded_new_reviews)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step


In [6]:
predictions

array([[0.9217708 ],
       [0.19113214],
       [0.5885772 ],
       [0.81452525],
       [0.300905  ],
       [0.94802755],
       [0.8816198 ],
       [0.5360552 ],
       [0.920713  ],
       [0.9473873 ]], dtype=float32)

In [7]:
# Classify new reviews using the optimal threshold
classified_reviews = (predictions >= threshold_object).astype(int)

# Print the predictions
for review, pred, prob in zip(movie_reviews, classified_reviews, predictions):
    label = "Positive" if pred == 1 else "Negative"
    print(f"Review: {review}\nProbability: {prob[0]:.4f}\nPrediction: {label}\n{'-'*50}")

Review: The film presents a deeply layered narrative that intertwines themes of loss, redemption, and human connection. While the pacing was deliberate, it allowed the emotional weight of each scene to resonate with the audience. The cinematography was breathtaking, capturing the stark beauty of the landscapes and the raw intensity of the characters' emotions. However, the overly ambiguous ending left some viewers dissatisfied, as it raised more questions than it answered.
Probability: 0.9218
Prediction: Positive
--------------------------------------------------
Review: A bold attempt at exploring societal issues through a dystopian lens, the movie falls short in execution. The script tries to juggle too many subplots, leaving several unresolved by the end. While the lead actor delivers a powerful performance, the supporting cast felt underutilized, and the soundtrack, though haunting, sometimes overshadowed critical dialogue. Nonetheless, the film's ambition deserves commendation, ev

# **Model Evaluation Summary**

## 1️. **Overall Model Performance**
- **Out of 10 reviews:**
  - ✅ **9 correctly classified** → **90% accuracy**
  - ❌ **1 misclassified**
- **Accuracy on this batch:**  
\[
\text{Accuracy} = \frac{9}{10} = 90\%
\]
- The model is **generalizing well** on unseen data.

---

## 2️. **Misclassified Review Analysis**
- **Review:**  
> *"Although the premise holds promise, the execution is far from perfect. The pacing is erratic, and the character arcs feel rushed, leaving little room for emotional engagement. The visuals are undoubtedly captivating, and the soundtrack elevates certain scenes, but the lack of coherence in the storytelling undermines the overall impact. It’s a film that has potential but fails to capitalize on it."*  

- **Model's Prediction:**  
  - **Probability:** 0.5361 → Predicted **Positive**
  - **True Sentiment:** **Negative**

### **Possible Reasons for Misclassification:**
1. **Ambiguous Language and Mixed Sentiment:**
   - The review contains both **positive and negative phrases**.
   - The model may have **weighed the positive-sounding phrases** more heavily.
2. **Threshold Limitation:**
   - With the **0.4138 threshold**, reviews near the threshold can still be misclassified.
